library

In [1]:
import argparse
import os
from PIL import Image
from tqdm.notebook import tqdm
import pickle

import torch
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"

parameter & utils

In [2]:
parser = argparse.ArgumentParser()

parser.add_argument('--sampling_frequency',type=int, default=20000)
parser.add_argument('--lr_length',type=int, default=1000, help='length of low-resolution signal')
parser.add_argument('--hr_length',type=int, default=10000)
parser.add_argument('--train_x_img_dir',type=str, default='./data/train/x')
parser.add_argument('--train_y_img_dir',type=str, default='./data/train/y')
parser.add_argument('--test_x_img_dir',type=str, default='./data/test/x')
parser.add_argument('--test_y_img_dir',type=str, default='./data/test/y')
parser.add_argument('--n_samples',type=int, default=2)
parser.add_argument('--patch_size',type=int, default=128)
parser.add_argument('--stride',type=int, default=96)

parser.add_argument('--epochs',type=int, default=10)
parser.add_argument('--lr',type=float, default=1e-4)
parser.add_argument('--early_stop',type=int, default=20, help='early stop_patience')
parser.add_argument('--batch_size',type=int, default=32)
parser.add_argument('--train_random_seed',type=int, default=42)
parser.add_argument('--test_random_seed',type=int, default=43)

opt = parser.parse_args('')

data preprocssing

In [9]:
class FFTProcessor():
    
    '''signal to FFT image'''
    
    def __init__(self, sampling_frequency, img_x_dir, img_y_dir, random_seed):
        self.fs = sampling_frequency
        self.img_x_dir = img_x_dir
        self.img_y_dir = img_y_dir
        self.random_seed = random_seed
        
    def generate_random_signal(self, length):
        t = np.linspace(0, length, int(self.fs), endpoint=False)
        x = np.zeros_like(t)
        
        num_components = np.random.randint(20, 50)  # 2~4
        for _ in range(num_components):
            A = np.random.uniform(0.5, 3.0)            # 진폭
            f = np.random.uniform(1, 20)               # 주파수 (Hz)
            phi = np.random.uniform(0, 2*np.pi)        # 위상
            x += A * np.sin(2 * np.pi * f * t + phi)
        return x

    def train_process_batch(self, n_samples, patch_size, stride):
        for sample_idx in range(n_samples):
            np.random.seed(self.random_seed)
            x = self.generate_random_signal(opt.lr_length)  # 이렇게 opt. 쓰는 것?? 별론가
            y = self.generate_random_signal(opt.hr_length)

            X = np.fft.fft(x)
            Y = np.fft.fft(y)
            #freqs = np.fft.fftfreq(len(x), d=1/self.fs)
            mag_X = np.abs(X)[:len(X)//2]
            mag_Y = np.abs(Y)[:len(Y)//2]
            tiled_X = np.tile(mag_X, (patch_size, 1))
            tiled_Y = np.tile(mag_Y, (patch_size, 1)) 
            
            plt.imsave(f"{self.img_x_dir}/fft_{sample_idx:03d}_wholeX.png", tiled_X, cmap='gray')
            plt.imsave(f"{self.img_x_dir}/fft_{sample_idx:03d}_wholeY.png", tiled_Y, cmap='gray')

            patch_idx = 0
            for i in range(0, tiled_X.shape[0] - patch_size + 1, stride):
                for j in range(0, tiled_X.shape[1] - patch_size + 1, stride):
                    patch = tiled_X[i:i+patch_size, j:j+patch_size]
                    plt.imsave(f"{self.img_x_dir}/fft_{sample_idx:03d}_{patch_idx:02d}.png", patch, cmap='gray')
                    patch_idx += 1
                    
            patch_idx = 0
            for i in range(0, tiled_Y.shape[0] - patch_size + 1, stride):
                for j in range(0, tiled_Y.shape[1] - patch_size + 1, stride):
                    patch = tiled_Y[i:i+patch_size, j:j+patch_size]
                    plt.imsave(f"{self.img_y_dir}/fft_{sample_idx:03d}_{patch_idx:02d}.png", patch, cmap='gray')
                    patch_idx += 1
                    
        

    def test_process_batch(self, n_samples):
        for idx in range(n_samples):
            np.random.seed(self.random_seed)
            x = self.generate_random_signal(opt.lr_length)
            y = self.generate_random_signal(opt.hr_length)

            X = np.fft.fft(x)
            Y = np.fft.fft(y)
            #freqs = np.fft.fftfreq(len(x), d=1/self.fs)
            mag_X = np.abs(X)[:len(X)//2]
            mag_Y = np.abs(Y)[:len(Y)//2]
            #tiled_X = np.tile(mag_X, (opt.patch_size, 1))
            tiled_X = np.tile(mag_X, (opt.hr_length, 1))
            tiled_Y = np.tile(mag_Y, (opt.hr_length, 1)) 

            plt.imsave(f"{self.img_x_dir}/fft_{idx:03d}.png", tiled_X, cmap='gray')
            plt.imsave(f"{self.img_y_dir}/fft_{idx:03d}.png", tiled_Y, cmap='gray')

In [73]:
def save_patch(x, img_dir, idx):
    
    plt.plot(x[:, 0], x[:, 1])
    plt.axis('off')     # 축 숨기기 (x, y 모두)
    plt.gca().spines['top'].set_visible(False)    # 테두리(스파인) 숨기기
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.savefig(f"{img_dir}/fft_{idx:03d}.png", bbox_inches = 'tight', transparent = True)
    plt.close()

In [81]:
#TEST

class FFTProcessor():
    
    '''signal to FFT image'''
    
    def __init__(self, sampling_frequency, img_x_dir, img_y_dir, random_seed):
        self.fs = sampling_frequency
        self.img_x_dir = img_x_dir
        self.img_y_dir = img_y_dir
        self.random_seed = random_seed
        
    def generate_random_signal(self, length):
        t = np.linspace(0, length, int(self.fs), endpoint=False)
        x = np.zeros_like(t)
        
        num_components = np.random.randint(20, 50)  # 2~4
        for _ in range(num_components):
            A = np.random.uniform(0.5, 3.0)            # 진폭
            f = np.random.uniform(1, 20)               # 주파수 (Hz)
            phi = np.random.uniform(0, 2*np.pi)        # 위상
            x += A * np.sin(2 * np.pi * f * t + phi)
        return x

    def train_process_batch(self, n_samples, patch_size, stride):
        for sample_idx in range(n_samples):
            np.random.seed(self.random_seed)
            x = self.generate_random_signal(opt.lr_length)  # 이렇게 opt. 쓰는 것?? 별론가
            y = self.generate_random_signal(opt.hr_length)

            X = np.fft.fft(x)
            Y = np.fft.fft(y)
            freqs = np.fft.fftfreq(len(X), d=1/self.fs)
            mag_X = np.abs(X)* 2 / len(X)
            mag_Y = np.abs(Y)* 2 / len(Y)
            freqs = np.fft.fftfreq(len(X), d=1/self.fs)
        
            half = int(len(X)/2)
            freq_mag_X = np.column_stack((freqs[:half], mag_X[:half]))
            freq_mag_Y = np.column_stack((freqs[:half], mag_Y[:half]))
            
            save_patch(freq_mag_X, self.img_x_dir, sample_idx)
            save_patch(freq_mag_Y, self.img_y_dir, sample_idx)

                
    def test_process_batch(self, n_samples):
        for idx in range(n_samples):
            np.random.seed(self.random_seed)
            x = self.generate_random_signal(opt.lr_length)
            y = self.generate_random_signal(opt.hr_length)

            X = np.fft.fft(x)
            Y = np.fft.fft(y)
            #freqs = np.fft.fftfreq(len(x), d=1/self.fs)
            mag_X = np.abs(X)[:len(X)//2]
            mag_Y = np.abs(Y)[:len(Y)//2]

            #plt.imsave(f"{self.img_x_dir}/fft_{idx:03d}.png", mag_X, cmap='gray')
            #plt.imsave(f"{self.img_y_dir}/fft_{idx:03d}.png", mag_Y, cmap='gray')
            

In [82]:
FFTProcessor(opt.sampling_frequency, opt.train_x_img_dir, opt.train_y_img_dir, opt.train_random_seed).train_process_batch(opt.n_samples, opt.patch_size, opt.stride)
FFTProcessor(opt.sampling_frequency, opt.test_x_img_dir, opt.test_y_img_dir, opt.test_random_seed).test_process_batch(opt.n_samples)

data load

In [165]:
class make_dataset():
    def __init__(self, train_x, train_y, test_x, test_y):
        self.train_x = train_x
        self.train_y = train_y
        self.test_x = test_x
        self.test_y = test_y
        
    def main(self):
        self.to_tensor()
        return self.train_dl, self.test_dl
                
    def to_tensor(self):
        # To tensor
        train_x = torch.tensor(self.train_x)
        train_y = torch.tensor(self.train_y)
        test_x = torch.tensor(self.test_x)
        test_y = torch.tensor(self.test_y)

        train = MyDataset(train_x, train_y)
        test = MyDataset(test_x, test_y)

        self.train_dl = DataLoader(train, opt.batch_size, shuffle=True)
        self.test_dl = DataLoader(test, opt.batch_size, shuffle=False)
    
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.x = self.x.unsqueeze(1)
        self.y = torch.argmax(y, 1)
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, ix):
        x = self.x[ix]
        y = self.y[ix]
        return x.to(device).float(), y.to(device).long()

In [171]:
train_x = []
for filename in os.listdir('./data/train/x'):
    img_path = os.path.join('./data/train/x', filename)
    img = Image.open(img_path).convert('RGB')
    img_array = np.array(img)
    train_x.append(img_array)
    
train_y = []
for filename in os.listdir('./data/train/y'):
    img_path = os.path.join('./data/train/y', filename)
    img = Image.open(img_path).convert('RGB')
    img_array = np.array(img)
    train_y.append(img_array)
    
test_x = []
for filename in os.listdir('./data/test/x'):
    img_path = os.path.join('./data/test/x', filename)
    img = Image.open(img_path).convert('RGB')
    img_array = np.array(img)
    test_x.append(img_array)
    
test_y = []
for filename in os.listdir('./data/test/y'):
    img_path = os.path.join('./data/test/y', filename)
    img = Image.open(img_path).convert('RGB')
    img_array = np.array(img)
    test_y.append(img_array)


c:\Users\limuri\anaconda3\envs\superi\Lib\site-packages\PIL\Image.py:3402: DecompressionBombWarning: Image size (100000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [172]:
train_dl, test_dl = make_dataset(train_x, train_y, test_x, test_y).main()

C:\Users\limuri\AppData\Local\Temp\ipykernel_6408\50614297.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:281.)
  train_x = torch.tensor(self.train_x)


model

In [174]:
class SRCNN(nn.Module):
    def __init__(self, num_channels=1): #흑백
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=9, padding = 9//2) #(H, W) 
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding = 5//2)
        self.conv3 = nn.Conv2d(32, num_channels, kernel_size=5, padding = 5//2)
        self.relu = nn.ReLU
        
        
    def forward(self, x):
        x=self.relu(self.conv1(x))
        x=self.relu(self.conv2(x))
        x=self.conv3(x)
        return x

train

In [183]:
def train(opt, train_dl):
    
    dict_result = {'train_loss': [],
                   'stop_point': 0
                    }
    
    best_loss = 10000
    early_stopping_count = 0
    
    #model = Model().to(device)
    model = SRCNN().to(device)
    loss_cross = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(list(model.parameters()), lr=opt.lr)
    #scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda epoch: opt.lamda ** epoch)
    
    pbar = tqdm(range(opt.epochs), unit = 'epoch')
    
    for epoch in pbar:
        iter_loss = []
        #iter_valid_loss = []
        
        # Training Model
        model.train()
        for ix, batch in enumerate(iter(train_dl)):
            x, y = batch
            pred = model(x)
            loss_y = loss_cross(pred, y)

            batch_loss = loss_y
            
            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()
            
            iter_loss.append(batch_loss.item())
            
            pbar.set_postfix({'train_loss' : loss_y.item()})

    #    scheduler.step()
        '''
        # Validation
        model.eval()
        with torch.no_grad():
            
            for ix, batch in enumerate(iter(valid_dl)):
                v_x, v_y = batch
                v_pred = model(v_x)
                v_loss_y = loss_cross(v_pred, v_y)
                
                iter_valid_loss.append(v_loss_y.item())
'''
        epoch_train = np.mean(iter_loss)
        #epoch_valid = np.mean(iter_valid_loss)

        #print( ' Epoch: {} / train_loss: {:.4f} / valid_loss: {:.4f}'.format(epoch+1, epoch_train, epoch_valid))
        print( ' Epoch: {} / train_loss: {:.4f} / valid_loss: {:.4f}'.format(epoch+1, epoch_train))
            
        ##################### result_save ############################           
        dict_result['train_loss'].append(epoch_train)
        #dict_result['valid_loss'].append(epoch_valid)
        ###############################################################
'''            
        if epoch_valid < best_loss:
            best_model = model
            early_stopping_count = 0
            torch.save(model, result_dir + '/model.pt')
            dict_result['stop_point'] = epoch+1
            print( ' >>> Best model save ! '.format(epoch+1, epoch_valid))
            
            best_loss = epoch_valid

        else:
            early_stopping_count = early_stopping_count + 1

        if early_stopping_count > opt.early_stop:
            break
           
    with open(result_dir + '/result.pickle', 'wb') as f:
        pickle.dump(dict_result, f)
        '''  

"            \n        if epoch_valid < best_loss:\n            best_model = model\n            early_stopping_count = 0\n            torch.save(model, result_dir + '/model.pt')\n            dict_result['stop_point'] = epoch+1\n            print( ' >>> Best model save ! '.format(epoch+1, epoch_valid))\n            \n            best_loss = epoch_valid\n\n        else:\n            early_stopping_count = early_stopping_count + 1\n\n        if early_stopping_count > opt.early_stop:\n            break\n           \n    with open(result_dir + '/result.pickle', 'wb') as f:\n        pickle.dump(dict_result, f)\n        "

In [ ]:
train_dl[]

In [184]:
train(opt, train_dl)

  0%|          | 0/10 [00:00<?, ?epoch/s]

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [206, 1, 128, 128, 3]

test